In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("Loan.csv")
data.dropna(inplace = True)

In [3]:
Y = data.iloc[:,12]
m = len(Y)
Y = Y.values

In [4]:
X = data.iloc[:,1:12]
print(X.head(6))

  Gender Married Dependents     Education Self_Employed  ApplicantIncome  \
1   Male     Yes          1      Graduate            No             4583   
2   Male     Yes          0      Graduate           Yes             3000   
3   Male     Yes          0  Not Graduate            No             2583   
4   Male      No          0      Graduate            No             6000   
5   Male     Yes          2      Graduate           Yes             5417   
6   Male     Yes          0  Not Graduate            No             2333   

   CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
1             1508.0       128.0             360.0             1.0   
2                0.0        66.0             360.0             1.0   
3             2358.0       120.0             360.0             1.0   
4                0.0       141.0             360.0             1.0   
5             4196.0       267.0             360.0             1.0   
6             1516.0        95.0             36

In [5]:
print(X['LoanAmount'].describe())

count    480.000000
mean     144.735417
std       80.508164
min        9.000000
25%      100.000000
50%      128.000000
75%      170.000000
max      600.000000
Name: LoanAmount, dtype: float64


In [6]:
print(X['Gender'].describe())

count      480
unique       2
top       Male
freq       394
Name: Gender, dtype: object


In [7]:
print(X['Married'].describe())

count     480
unique      2
top       Yes
freq      311
Name: Married, dtype: object


In [8]:
print(X['Loan_Amount_Term'].describe())

count    480.000000
mean     342.050000
std       65.212401
min       36.000000
25%      360.000000
50%      360.000000
75%      360.000000
max      480.000000
Name: Loan_Amount_Term, dtype: float64


In [9]:
print(X['Credit_History'].describe())

count    480.000000
mean       0.854167
std        0.353307
min        0.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: Credit_History, dtype: float64


In [10]:
print(X['CoapplicantIncome'].describe())

count      480.000000
mean      1581.093583
std       2617.692267
min          0.000000
25%          0.000000
50%       1084.500000
75%       2253.250000
max      33837.000000
Name: CoapplicantIncome, dtype: float64


In [11]:
print(X['Education'].describe())

count          480
unique           2
top       Graduate
freq           383
Name: Education, dtype: object


In [12]:
print(X['ApplicantIncome'].describe())

count      480.000000
mean      5364.231250
std       5668.251251
min        150.000000
25%       2898.750000
50%       3859.000000
75%       5852.500000
max      81000.000000
Name: ApplicantIncome, dtype: float64


In [13]:
print(X['Dependents'].describe())

count     480
unique      4
top         0
freq      274
Name: Dependents, dtype: object


In [14]:
print(X['Self_Employed'].describe())

count     480
unique      2
top        No
freq      414
Name: Self_Employed, dtype: object


In [15]:
X['Property_Area'].replace(('Rural', 'Semiurban', 'Urban'), (1, 2, 3),inplace=True)
X = pd.get_dummies(X)
X["Total_Income"] = X["ApplicantIncome"] + X["CoapplicantIncome"]
X["Loan/Term"] = X["LoanAmount"]*1000/X["Loan_Amount_Term"]
X["Loan/Income"] = X["LoanAmount"]*1000/X["Total_Income"]
X["Balance"] = X["Total_Income"] - X["Loan/Term"]*0.06*12
print(X.head(6))
X = X.values
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(X)
X = imp.transform(X)

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
1             4583             1508.0       128.0             360.0   
2             3000                0.0        66.0             360.0   
3             2583             2358.0       120.0             360.0   
4             6000                0.0       141.0             360.0   
5             5417             4196.0       267.0             360.0   
6             2333             1516.0        95.0             360.0   

   Credit_History  Property_Area  Gender_Female  Gender_Male  Married_No  \
1             1.0              1              0            1           0   
2             1.0              3              0            1           0   
3             1.0              3              0            1           0   
4             1.0              3              0            1           1   
5             1.0              3              0            1           0   
6             1.0              3              

In [16]:
m,n = X.shape
print("Number of training examples =",m)
print("Number of features =",n)

Number of training examples = 480
Number of features = 22


In [17]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C = 0.1,max_iter = 10000,solver = 'newton-cg',tol = 1e-0)
lr.fit(X,Y)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='newton-cg',
          tol=1.0, verbose=0, warm_start=False)

In [18]:
score = lr.score(X, Y)
print("Training set accuracy = ",score*100)

Training set accuracy =  81.04166666666667


Test set accuracy = 79.1667

In [19]:
dat = pd.read_csv("Test.csv")
X1 = dat.iloc[:,1:12]
X2 = dat.iloc[:,0]
X1['Property_Area'].replace(('Rural', 'Semiurban', 'Urban'), (1, 2, 3),inplace=True)
X1 = pd.get_dummies(X1)
X1["Total_Income"] = X1["ApplicantIncome"] + X1["CoapplicantIncome"]
X1["Loan/Term"] = X1["LoanAmount"]*1000/X1["Loan_Amount_Term"]
X1["Loan/Income"] = X1["LoanAmount"]*1000/X1["Total_Income"]
X1["Balance"] = X1["Total_Income"] - X1["Loan/Term"]*0.06*12
X1 = X1.values
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(X1)
X1 = imp.transform(X1)

In [20]:
predict = lr.predict(X1)

In [21]:
pd.DataFrame({'Loan_ID':X2,'Loan_Status':predict}).to_csv('Submission.csv',index=False)